---
title: "Cortex Forecast Example"
date: "08/20/2024"
format:
    html:
        code-fold: False
        output: False
---

# Overview

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
#| hide
from nbdev.showdoc import *

# Example Useage

In [95]:
#| skip
from cortex_forecast.forecast import SnowflakeForecast
from snowflake.snowpark.version import VERSION
import os

In [97]:
#| skip
forecast_model = SnowflakeMLForecast(
    config_file='./forecast/files/yaml/forecast_config.yaml',
    connection_config={
        'user': os.getenv('SNOWFLAKE_USER'),
        'password': os.getenv('SNOWFLAKE_PASSWORD'),
        'account': os.getenv('SNOWFLAKE_ACCOUNT'),
        'database': 'CORTEX',
        'warehouse': 'CORTEX_WH',
        'schema': 'DEV',
        'role': 'CORTEX_USER_ROLE'  # Use the desired role
    }
)

snowflake_environment = forecast_model.session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(forecast_model.session.get_current_role()))
print('Database                    : {}'.format(forecast_model.session.get_current_database()))
print('Schema                      : {}'.format(forecast_model.session.get_current_schema()))
print('Warehouse                   : {}'.format(forecast_model.session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0], snowpark_version[1], snowpark_version[2]))

# Create Training Data
training_days = 60
predicted_days = 30

forecast_model.session.sql(f'''CREATE OR REPLACE TABLE storage_usage_train AS
    SELECT 
        TO_TIMESTAMP_NTZ(usage_date) AS usage_date,
        storage_bytes / POWER(1024, 3) AS storage_gb
    FROM snowflake.account_usage.storage_usage
    WHERE TO_TIMESTAMP_NTZ(usage_date) < DATEADD(day, -{training_days}, CURRENT_DATE())
''').collect()
forecast_model.session.sql('SELECT * FROM storage_usage_train LIMIT 10').collect()


Connection Established with the following parameters:
User                        : JD_SERVICE_ACCOUNT_ADMIN
Role                        : "CORTEX_USER_ROLE"
Database                    : "CORTEX"
Schema                      : "DEV"
Warehouse                   : "CORTEX_WH"
Snowflake version           : 8.31.1
Snowpark for Python version : 1.19.0
Creating tag: CREATE OR REPLACE TAG environment COMMENT = 'Specifies the production';
Creating tag: CREATE OR REPLACE TAG team COMMENT = 'Specifies the data_science';
Step 1/4: Creating training table...
Step 2/4: Creating forecast model...
Generated SQL:

        CREATE OR REPLACE SNOWFLAKE.ML.FORECAST my_forecast_model_20240820_cttyu(
            INPUT_DATA => SYSTEM$REFERENCE('table', 'storage_usage_train'),
            TIMESTAMP_COLNAME => 'usage_date',
            TARGET_COLNAME => 'storage_gb',
        CONFIG_OBJECT => OBJECT_CONSTRUCT('evaluation_config', OBJECT_CONSTRUCT('n_splits', 5, 'test_size', 30, 'gap', 0, 'prediction_interval',

In [ ]:
# Run Forecast
forecast_model = SnowflakeMLForecast(
    config_file='./forecast/files/yaml/storage_forecast.yaml',
    connection_config={
        'user': os.getenv('SNOWFLAKE_USER'),
        'password': os.getenv('SNOWFLAKE_PASSWORD'),
        'account': os.getenv('SNOWFLAKE_ACCOUNT'),
        'database': 'CORTEX',
        'warehouse': 'CORTEX_WH',
        'schema': 'DEV',
        'role': 'CORTEX_USER_ROLE'
    }
)
forecast_data = forecast_model.create_and_run_forecast()
forecast_data

In [100]:
#| hide
import nbdev; nbdev.nbdev_export()